# Kaggle GAN Getting Started - 提交Notebook

本notebook用于在Kaggle平台上生成Monet风格的图像并创建提交文件。

## 📋 使用说明

1. **上传模型**: 将训练好的模型上传到Kaggle Dataset
2. **修改路径**: 更新下面的MODEL_PATH变量
3. **运行**: 按顺序运行所有cells
4. **下载**: 下载生成的images.zip文件

## ⚠️ 重要提示

- 确保模型路径正确
- 生成的图像数量应该在7000-10000张之间
- 图像格式为JPG，尺寸256x256


## 1. 导入库和设置


In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import re
from tqdm import tqdm
import PIL
import shutil

# 设置TensorFlow日志级别
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {tf.config.list_physical_devices('GPU')}")

# 配置参数
HEIGHT, WIDTH, CHANNELS = 256, 256, 3
AUTO = tf.data.experimental.AUTOTUNE

print("✅ 库导入完成")


## 2. 配置模型路径

**⚠️ 重要：请修改MODEL_PATH为你上传的模型路径！**


In [ ]:
# ===== 请修改为你的模型路径 =====
MODEL_PATH = "../input/your-model-dataset/monet_generator.h5"
# =================================

print(f"模型路径: {MODEL_PATH}")

# 检查模型文件是否存在
if os.path.exists(MODEL_PATH):
    print("✅ 模型文件存在")
else:
    print("❌ 模型文件不存在，请检查路径！")
    print("请确保：")
    print("1. 已将模型上传到Kaggle Dataset")
    print("2. 已正确设置MODEL_PATH变量")
    print("3. 模型文件名正确")


## 3. 定义辅助函数


In [ ]:
def count_data_items(filenames):
    """计算TFRecord文件中的数据项数量"""
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)

def decode_image(image):
    """解码JPEG图像并归一化到[-1, 1]"""
    image = tf.image.decode_jpeg(image, channels=CHANNELS)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [HEIGHT, WIDTH, CHANNELS])
    return image

def read_tfrecord(example):
    """读取TFRecord示例"""
    tfrecord_format = {
        'image_name': tf.io.FixedLenFeature([], tf.string),
        'image': tf.io.FixedLenFeature([], tf.string),
        'target': tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    image = decode_image(example['image'])
    return image

def load_dataset(filenames):
    """从TFRecord文件加载数据集"""
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    return dataset

print("✅ 辅助函数定义完成")


## 4. 加载数据


In [ ]:
# 获取数据路径
GCS_PATH = "/kaggle/input/gan-getting-started"
PHOTO_FILENAMES = tf.io.gfile.glob(str(GCS_PATH + '/photo_tfrec/*.tfrec'))

# 计算和加载数据
n_photo_samples = count_data_items(PHOTO_FILENAMES)
photo_ds = load_dataset(PHOTO_FILENAMES)

print(f"找到 {len(PHOTO_FILENAMES)} 个Photo TFRecord文件")
print(f"总Photo样本数: {n_photo_samples}")

# 配置数据集
photo_ds = photo_ds.batch(1)  # 单张图像批次

print("✅ 数据加载完成")


## 5. 加载模型


In [ ]:
# 加载训练好的Monet生成器
print(f"正在加载模型: {MODEL_PATH}")
try:
    loaded_model = tf.keras.models.load_model(MODEL_PATH)
    print("✅ 模型加载成功！")
    print(f"模型输入形状: {loaded_model.input_shape}")
    print(f"模型输出形状: {loaded_model.output_shape}")
except Exception as e:
    print(f"❌ 模型加载失败: {e}")
    print("请检查MODEL_PATH变量是否正确")


## 6. 生成Monet风格图像


In [ ]:
def predict_and_save(path, input_ds, generator_model):
    """生成图像并保存到磁盘"""
    i = 1
    total = sum(1 for _ in input_ds)
    
    for img in tqdm(input_ds, total=total, desc="生成图像"):
        # 生成预测
        prediction = generator_model(img, training=False)[0].numpy()
        
        # 从[-1, 1]缩放到[0, 255]
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
        
        # 保存为JPEG
        im = PIL.Image.fromarray(prediction)
        im.save(os.path.join(path, "images", f"{i}.jpg"))
        i += 1
    
    return i - 1

# 创建输出目录
output_dir = "./images"
os.makedirs(output_dir, exist_ok=True)

# 生成图像
print("开始生成Monet风格图像...")
num_generated = predict_and_save(
    path="./",
    input_ds=photo_ds,
    generator_model=loaded_model
)

print(f"\n✅ 生成了 {num_generated} 张图像")


## 7. 创建提交ZIP文件


In [ ]:
# 创建ZIP压缩包
images_path = os.path.join("./", 'images')
zip_path = shutil.make_archive(images_path, 'zip', images_path)

# 统计信息
num_files = len([name for name in os.listdir(images_path) if os.path.isfile(os.path.join(images_path, name))])
zip_size = os.path.getsize(zip_path) / (1024 * 1024)  # 大小（MB）

print(f"\n{'='*50}")
print(f"提交摘要:")
print(f"  生成图像数: {num_files}")
print(f"  ZIP文件: {zip_path}")
print(f"  ZIP大小: {zip_size:.2f} MB")
print(f"{'='*50}")

# 验证
if num_files < 7000:
    print("⚠️ 警告: 生成图像少于7000张！")
elif num_files > 10000:
    print("⚠️ 警告: 生成图像超过10000张！")
else:
    print("✅ 图像数量符合要求 (7000-10000张)")

print(f"\n🎉 提交文件已创建: {zip_path}")
print("请下载此文件并上传到竞赛页面")


## 完成！

### 📊 提交总结

- ✅ 模型加载成功
- ✅ 图像生成完成
- ✅ ZIP文件已创建
- ✅ 文件数量验证通过

### 🎯 下一步

1. **下载文件**: 点击左侧文件列表中的 `images.zip`
2. **上传提交**: 将ZIP文件上传到竞赛页面
3. **等待结果**: 等待Kaggle评估你的提交

### 💡 提示

- 如果图像数量不符合要求，请检查数据加载是否正确
- 如果生成质量不佳，可能需要重新训练模型
- 可以尝试不同的模型参数来改善效果

**祝你竞赛成功！** 🎨🏆
